### 3D Face Reconstruction without Extrinsic Parameters

In [3]:
import numpy as np
import cv2 as cv
import open3d as o3d
from numba import njit, prange

@njit(parallel=True)
def fast_depth_filter(points):
    valid = np.empty(len(points), dtype=np.bool_)
    for i in prange(len(points)):
        z = points[i, 2]
        valid[i] = 0.2 < z < 50.0
    return valid

def get_colors(img, points_2d):
    h, w = img.shape[:2]
    x = np.clip(points_2d[:, 0].round(), 0, w - 1).astype(int)
    y = np.clip(points_2d[:, 1].round(), 0, h - 1).astype(int)
    return img[y, x][:, None] / 255.0  # Grayscale to single-channel float

def optimized_3d_reconstruction():
    # Load stereo grayscale images
    img_left = cv.imread("5/Camera_Left.png", cv.IMREAD_GRAYSCALE)
    img_right = cv.imread("5/Camera_Front.png", cv.IMREAD_GRAYSCALE)
    if img_left is None or img_right is None:
        raise FileNotFoundError("Stereo images not found")

    # Intrinsic camera matrix
    K = np.array([[2666.67, 0, 960.0], [0, 2250.0, 540.0], [0, 0, 1]])

    # Feature detection
    sift = cv.SIFT_create(nfeatures=500000, contrastThreshold=0.002, edgeThreshold=10, nOctaveLayers=4, sigma=1.2)
    kpL, desL = sift.detectAndCompute(img_left, None)
    kpR, desR = sift.detectAndCompute(img_right, None)

    # Matching with Lowe's ratio test
    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(desL, desR, k=2)
    good = [m for m, n in matches if m.distance < 0.9 * n.distance]
    print(f"Good matches found: {len(good)}")
    if len(good) < 2000:
        raise ValueError("Insufficient good matches")

    # Extract matched points
    ptsL = np.float32([kpL[m.queryIdx].pt for m in good])
    ptsR = np.float32([kpR[m.trainIdx].pt for m in good])

    # Estimate essential matrix and camera pose
    E, mask = cv.findEssentialMat(ptsL, ptsR, K, method=cv.RANSAC, prob=0.999, threshold=1.0, maxIters=2000)
    if E is None or mask.sum() < 500:
        raise ValueError("Pose estimation failed")
    _, R, t, mask_pose = cv.recoverPose(E, ptsL, ptsR, K, mask=mask)

    # Filter inliers
    mask_final = mask.ravel() == 1
    ptsL, ptsR = ptsL[mask_final], ptsR[mask_final]

    # Triangulate 3D points
    P0 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
    P1 = K @ np.hstack([R, t])
    pts4d = cv.triangulatePoints(P0, P1, ptsL.T, ptsR.T)
    points3d = (pts4d[:3] / pts4d[3]).T

    # Depth filter
    valid_mask = fast_depth_filter(points3d)
    points3d = points3d[valid_mask]
    ptsL_valid = ptsL[valid_mask]

    # Use image color (grayscale) from left image
    colors = get_colors(img_left, ptsL_valid)
    colors = np.repeat(colors, 3, axis=1)  # Convert to RGB-like shape

    # Build and clean point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    # pcd = pcd.voxel_down_sample(voxel_size=0.005)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=3.0)

    # Save and report
    o3d.io.write_point_cloud("Reconstruction_pose.ply", pcd, write_ascii=False, compressed=True)
    print(f"Reconstructed and saved {len(pcd.points)} high-quality 3D points.")

if __name__ == "__main__":
    optimized_3d_reconstruction()


Good matches found: 11865
Reconstructed and saved 10007 high-quality 3D points.


### Add Texiture

In [1]:
import numpy as np
import cv2 as cv
import open3d as o3d
from numba import njit, prange

@njit(parallel=True)
def fast_depth_filter(points):
    valid = np.empty(len(points), dtype=np.bool_)
    for i in prange(len(points)):
        z = points[i, 2]
        valid[i] = 0.05 < z < 100.0
    return valid

def get_colors(img, points_2d):
    h, w = img.shape[:2]
    x = np.clip(points_2d[:, 0].round(), 0, w - 1).astype(int)
    y = np.clip(points_2d[:, 1].round(), 0, h - 1).astype(int)
    return img[y, x][:, None] / 255.0  # Grayscale to single-channel float

def optimized_3d_reconstruction():
    # Load stereo grayscale images
    img_left = cv.imread("5/Camera_Left.png", cv.IMREAD_GRAYSCALE)
    img_right = cv.imread("5/Camera_Front.png", cv.IMREAD_GRAYSCALE)
    if img_left is None or img_right is None:
        raise FileNotFoundError("Stereo images not found")

    # Intrinsic camera matrix
    K = np.array([[2666.67, 0, 960.0], [0, 2250.0, 540.0], [0, 0, 1]])

    # Feature detection
    sift = cv.SIFT_create(nfeatures=500000, contrastThreshold=0.001, edgeThreshold=5, nOctaveLayers=4, sigma=1)
    kpL, desL = sift.detectAndCompute(img_left, None)
    kpR, desR = sift.detectAndCompute(img_right, None)

    # Matching with Lowe's ratio test
    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(desL, desR, k=2)
    good = [m for m, n in matches if m.distance < 0.9 * n.distance]
    print(f"Good matches found: {len(good)}")
    if len(good) < 2000:
        raise ValueError("Insufficient good matches")

    # Extract matched points
    ptsL = np.float32([kpL[m.queryIdx].pt for m in good])
    ptsR = np.float32([kpR[m.trainIdx].pt for m in good])

    # Estimate essential matrix and camera pose
    E, mask = cv.findEssentialMat(ptsL, ptsR, K, method=cv.RANSAC, prob=0.999, threshold=0.5, maxIters=5000)
    if E is None or mask.sum() < 500:
        raise ValueError("Pose estimation failed")
    _, R, t, mask_pose = cv.recoverPose(E, ptsL, ptsR, K, mask=mask)

    # Filter inliers
    mask_final = mask.ravel() == 1
    ptsL, ptsR = ptsL[mask_final], ptsR[mask_final]

    # Triangulate 3D points
    P0 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
    P1 = K @ np.hstack([R, t])
    pts4d = cv.triangulatePoints(P0, P1, ptsL.T, ptsR.T)
    points3d = (pts4d[:3] / pts4d[3]).T

    # Depth filter
    valid_mask = fast_depth_filter(points3d)
    points3d = points3d[valid_mask]
    ptsL_valid = ptsL[valid_mask]

    # Use image color (grayscale) from left image
    colors = get_colors(img_left, ptsL_valid)
    colors = np.repeat(colors, 3, axis=1)  # Convert to RGB-like shape

    # Build and clean point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d)
    pcd.colors = o3d.utility.Vector3dVector(colors)
   # pcd = pcd.voxel_down_sample(voxel_size=0.005)
    #pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=5, std_ratio=3.0)

    # Save and report
    o3d.io.write_point_cloud("reconstruction_refined.ply", pcd, write_ascii=False, compressed=True)
    print(f"Reconstructed and saved {len(pcd.points)} high-quality 3D points.")


    # === Step 1: Use RGB image for real color ===
    img_left_rgb = cv.imread("5/Camera_Left.png", cv.IMREAD_COLOR)
    if img_left_rgb is None:
        raise FileNotFoundError("Left RGB image not found")
    img_left_rgb = cv.cvtColor(img_left_rgb, cv.COLOR_BGR2RGB)

    def get_rgb_colors(img_color, points_2d):
        h, w = img_color.shape[:2]
        x = np.clip(points_2d[:, 0].round(), 0, w - 1).astype(int)
        y = np.clip(points_2d[:, 1].round(), 0, h - 1).astype(int)
        return img_color[y, x] / 255.0

    colors = get_rgb_colors(img_left_rgb, ptsL_valid)

    # === Step 2: Build point cloud with real color ===
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # === Step 3: Clean point cloud ===
    pcd = pcd.voxel_down_sample(voxel_size=0.005)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(k=20)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.0)

    # === Step 4: Create mesh from point cloud ===
    print("Generating mesh using Poisson reconstruction...")
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=13)
    mesh.compute_vertex_normals()

    # === Step 5: Crop mesh to remove floating artifacts ===
    bbox = pcd.get_axis_aligned_bounding_box()
    mesh = mesh.crop(bbox)

    # === Step 6: Color vertices using image ===
    verts = np.asarray(mesh.vertices)
    verts_proj = (K @ verts.T).T
    verts_proj = verts_proj[:, :2] / verts_proj[:, 2:3]  # Normalize to pixel coords

    mesh_colors = get_rgb_colors(img_left_rgb, verts_proj)
    mesh.vertex_colors = o3d.utility.Vector3dVector(mesh_colors)

    # === Step 7: Save final mesh ===
    o3d.io.write_triangle_mesh("textured_mesh_pose.ply", mesh, write_ascii=False, compressed=True)
    print(f"Mesh saved with texture: {len(mesh.vertices)} vertices, {len(mesh.triangles)} faces.")
if __name__ == "__main__":
    optimized_3d_reconstruction()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Good matches found: 15950
Reconstructed and saved 12465 high-quality 3D points.
Generating mesh using Poisson reconstruction...
Mesh saved with texture: 38350 vertices, 76363 faces.


### Pose Estimation and Evaluation of Reconstructed 3D Face

In [2]:
import numpy as np
import cv2 as cv
import open3d as o3d
from numba import njit, prange

@njit(parallel=True)
def fast_depth_filter(points):
    valid = np.empty(len(points), dtype=np.bool_)
    for i in prange(len(points)):
        z = points[i, 2]
        valid[i] = 0.2 < z < 50.0
    return valid

def get_colors(img, points_2d):
    h, w = img.shape[:2]
    x = np.clip(points_2d[:, 0].round(), 0, w - 1).astype(int)
    y = np.clip(points_2d[:, 1].round(), 0, h - 1).astype(int)
    return img[y, x][:, None] / 255.0

def compute_reprojection_error(points3d, pts2d, P):
    points3d_hom = np.hstack((points3d, np.ones((len(points3d), 1))))
    proj = (P @ points3d_hom.T).T
    proj = proj[:, :2] / proj[:, 2:]
    errors = np.linalg.norm(proj - pts2d, axis=1)
    return np.mean(errors)

def optimized_3d_reconstruction():
    img_left = cv.imread("5/Camera_Left.png", cv.IMREAD_GRAYSCALE)
    img_right = cv.imread("5/Camera_Front.png", cv.IMREAD_GRAYSCALE)
    if img_left is None or img_right is None:
        raise FileNotFoundError("Stereo images not found")

    K = np.array([[2666.67, 0, 960.0], [0, 2250.0, 540.0], [0, 0, 1]])

    sift = cv.SIFT_create(nfeatures=500000, contrastThreshold=0.002, edgeThreshold=10, nOctaveLayers=4, sigma=1.2)
    kpL, desL = sift.detectAndCompute(img_left, None)
    kpR, desR = sift.detectAndCompute(img_right, None)

    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(desL, desR, k=2)
    good = [m for m, n in matches if m.distance < 0.9 * n.distance]
    print(f"Good matches found: {len(good)}")
    if len(good) < 2000:
        raise ValueError("Insufficient good matches")

    ptsL = np.float32([kpL[m.queryIdx].pt for m in good])
    ptsR = np.float32([kpR[m.trainIdx].pt for m in good])

    E, mask = cv.findEssentialMat(ptsL, ptsR, K, method=cv.RANSAC, prob=0.999, threshold=1.0, maxIters=2000)
    inlier_ratio = mask.sum() / len(mask)
    print(f"Essential Matrix Inliers: {inlier_ratio:.1%}")

    if E is None or mask.sum() < 500:
        raise ValueError("Pose estimation failed")

    _, R, t, _ = cv.recoverPose(E, ptsL, ptsR, K, mask=mask)

    mask_final = mask.ravel() == 1
    ptsL, ptsR = ptsL[mask_final], ptsR[mask_final]

    P0 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
    P1 = K @ np.hstack([R, t])
    pts4d = cv.triangulatePoints(P0, P1, ptsL.T, ptsR.T)
    points3d = (pts4d[:3] / pts4d[3]).T

    # Average Reprojection Error
    err0_mean = compute_reprojection_error(points3d, ptsL, P0)
    err1_mean = compute_reprojection_error(points3d, ptsR, P1)
    avg_reproj_error = (err0_mean + err1_mean) / 2
    print(f"Average Reprojection Error: {avg_reproj_error:.4f}px")

    # Triangulation Angles
    C0 = np.zeros(3)
    C1 = -R.T @ t.reshape(3)
    vec1 = points3d - C0
    vec2 = points3d - C1
    norms_prod = np.linalg.norm(vec1, axis=1) * np.linalg.norm(vec2, axis=1)
    cos_angles = np.sum(vec1 * vec2, axis=1) / np.maximum(norms_prod, 1e-10)
    angles_deg = np.degrees(np.arccos(np.clip(cos_angles, -1, 1)))
    print(f"Triangulation Angles: Min={np.min(angles_deg):.2f}°, Mean={np.mean(angles_deg):.2f}°, Max={np.max(angles_deg):.2f}°")

    # Depth Filter
    valid_mask = fast_depth_filter(points3d)
    points3d = points3d[valid_mask]
    ptsL_valid = ptsL[valid_mask]

    # Color Mapping
    colors = get_colors(img_left, ptsL_valid)
    colors = np.repeat(colors, 3, axis=1)

    # Point Cloud Creation
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    o3d.io.write_point_cloud("reconstruction_Pose.ply", pcd, write_ascii=False, compressed=True)
    print(f"Saved point cloud with {len(pcd.points)} high-quality 3D points.")

if __name__ == "__main__":
    optimized_3d_reconstruction()


Good matches found: 11865
Essential Matrix Inliers: 84.5%
Average Reprojection Error: 0.1090px
Triangulation Angles: Min=3.69°, Mean=5.56°, Max=16.70°
Saved point cloud with 10025 high-quality 3D points.
